В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

In [324]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC

In [325]:
from warnings import filterwarnings
filterwarnings('ignore')

In [326]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None, names=['Text', 'result']) 
test = pd.read_csv('products_sentiment_test.tsv', sep='\t', index_col = ['Id'])

In [343]:
def text_classifier(vectorizer, classifier):
    return Pipeline([("vectorizer", vectorizer),("classifier", classifier)])

In [344]:
def cut_string(v):
    return str(v).rsplit('.', 1)[1].replace(">", "").replace('\'', '')

In [329]:
vect = [TfidfVectorizer, CountVectorizer, HashingVectorizer]
classif = [LogisticRegression, MultinomialNB, SGDClassifier, LinearSVC]

In [330]:
df = pd.DataFrame(columns=['vectorizer', 'transformer', 'classifier', 'score'])

In [341]:
i = 0
for v in vect:
    for c in classif:
        i += 1
        pipeline = text_classifier(v(ngram_range=(1, 4),stop_words='english', binary=True), c())

        a = [round(cross_val_score(pipeline, train.Text, train.result, cv=5).mean(), 4)]
        df.loc[i] =  (cut_string(v), 'default', cut_string(c), a)

In [342]:
df.sort_values(by=['score'], ascending=False)

,vectorizer,transformer,classifier,score
4,TfidfVectorizer,default,LinearSVC,[0.751]
6,CountVectorizer,default,MultinomialNB,[0.75]
7,CountVectorizer,default,SGDClassifier,[0.748]
12,HashingVectorizer,default,LinearSVC,[0.7465]
5,CountVectorizer,default,LogisticRegression,[0.7405]
11,HashingVectorizer,default,SGDClassifier,[0.7395]
3,TfidfVectorizer,default,SGDClassifier,[0.737]
8,CountVectorizer,default,LinearSVC,[0.7355]
1,TfidfVectorizer,default,LogisticRegression,[0.702]
9,HashingVectorizer,default,LogisticRegression,[0.6885]


In [345]:
#Вот наш победитель

In [337]:
pipeline = text_classifier(HashingVectorizer(ngram_range=(4, 8),stop_words='english', analyzer='char_wb', binary=True),
                           LinearSVC())

In [338]:
_ = pipeline.fit(train.Text, train.result)

In [339]:
test['y'] = pipeline.predict(test.text)

In [340]:
test.to_csv(index=True, columns = ['y'], header=True,
            path_or_buf='/home/k3di/Coursera_Yandex/Course 6/answer.csv')